In [1]:
import sys
import os

# Get current working directory (e.g., notebook location)
notebook_dir = os.getcwd()

# Construct the path to the correct directory
parent_dir = os.path.abspath(os.path.join(notebook_dir, '../src'))

# Add it to sys.path
sys.path.insert(0, parent_dir)

# Now you can import the module
from functions.loading import load_csv
from functions.merging import merge_dataframes
from functions.imputation import impute_with_mean
from functions.remerge import remerge
from functions.train import train_model
from functions.test import test_model, get_min_from_csv
from functions.submission import create_submission
from functions.encoding import removal_nonnumeric_columns
from functions.split import split_data
from functions.predict import make_prediction
import pandas as pd
import numpy as np

In [2]:
# Read data 
train_features = load_csv("../data/01_raw/dengue_features_train.csv")
train_labels = load_csv("../data/01_raw/dengue_labels_train.csv")
test_features = load_csv("../data/01_raw/dengue_features_test.csv")

df = merge_dataframes(train_features, train_labels, test_features, on=["city", "weekofyear", "year"])

df.head()

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases
0,sj,1990,18,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,...,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0,4.0
1,sj,1990,19,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,...,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6,5.0
2,sj,1990,20,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,...,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4,4.0
3,sj,1990,21,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,...,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0,3.0
4,sj,1990,22,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,...,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8,6.0


In [3]:
df.describe()

,year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,...,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases
count,1872.000000,1872.000000,1635.000000,1809.000000,1849.000000,1849.000000,1857.000000,1860.000000,1860.000000,1860.000000,...,1860.000000,1857.000000,1860.000000,1860.000000,1817.000000,1817.000000,1849.000000,1849.000000,1845.000000,1456.000000
mean,2003.194979,26.489316,0.138588,0.129713,0.204663,0.202174,44.109278,298.727770,299.253955,295.284823,...,82.237158,44.109278,16.786638,4.952903,27.226651,8.004112,32.470795,22.160790,38.201897,24.675137
std,6.292268,15.006319,0.146413,0.125082,0.075064,0.085768,42.065868,1.387410,1.272510,1.528046,...,7.203803,42.065868,1.547336,3.545887,1.281254,2.205734,1.950483,1.613325,44.961360,43.596000
min,1990.000000,1.000000,-0.463400,-0.456100,-0.015533,-0.063457,0.000000,294.554286,294.892857,289.642857,...,57.787143,0.000000,11.715714,1.357143,21.400000,4.042857,26.700000,14.200000,0.000000,0.000000
25%,1999.000000,13.750000,0.035300,0.041225,0.153457,0.141014,9.430000,297.678929,298.276786,294.164643,...,77.214643,9.430000,15.590714,2.357143,26.328571,6.357143,31.100000,21.100000,8.800000,5.000000
50%,2004.000000,26.000000,0.127317,0.115200,0.197029,0.188229,36.640000,298.617143,299.300000,295.667143,...,80.307143,36.640000,17.137143,2.871429,27.433333,7.200000,32.800000,22.200000,23.800000,12.000000
75%,2008.000000,39.000000,0.252225,0.223360,0.250857,0.248086,67.500000,299.891429,300.271429,296.502857,...,86.799286,67.500000,18.028929,7.757143,28.200000,9.625000,33.900000,23.300000,51.500000,28.000000
max,2013.000000,53.000000,0.508357,0.649000,0.538314,0.546017,390.600000,302.200000,303.328571,298.450000,...,98.610000,390.600000,20.461429,16.028571,30.800000,15.800000,42.200000,26.700000,543.300000,461.000000


# Cyclical Encoding

In [4]:
# Ensure date is in datetime format
df['week_start_date'] = pd.to_datetime(df['week_start_date'])

# Extract date components
df['month'] = df['week_start_date'].dt.month
df['weekofyear'] = df['week_start_date'].dt.isocalendar().week
df['dayofweek'] = df['week_start_date'].dt.dayofweek

# Cyclical encoding
def add_cyclical_features(df, col, period):
    df[f'{col}_sin'] = np.sin(2 * np.pi * df[col] / period)
    df[f'{col}_cos'] = np.cos(2 * np.pi * df[col] / period)

add_cyclical_features(df, 'month', 12)
add_cyclical_features(df, 'weekofyear', 52)
add_cyclical_features(df, 'dayofweek', 7)

# Lag Features

In [ ]:
# Columns to create lag/rolling features for
lag_features = [
    'station_max_temp_c', 'station_min_temp_c', 'station_avg_temp_c', 'station_precip_mm',
    'station_diur_temp_rng_c', 'precipitation_amt_mm', 'reanalysis_sat_precip_amt_mm',
    'reanalysis_dew_point_temp_k', 'reanalysis_air_temp_k', 'reanalysis_relative_humidity_percent',
    'reanalysis_specific_humidity_g_per_kg', 'reanalysis_precip_amt_kg_per_m2',
    'reanalysis_max_air_temp_k', 'reanalysis_min_air_temp_k', 'reanalysis_avg_temp_k',
    'reanalysis_tdtr_k', 'ndvi_se', 'ndvi_sw', 'ndvi_ne', 'ndvi_nw'
]

lags = [1, 2, 3]
roll_windows = [3, 5]

# Sort by city and time
df = df.sort_values(['city', 'week_start_date'])

# Apply lags and rolling means per city
for col in lag_features:
    for lag in lags:
        df[f'{col}_lag{lag}'] = df.groupby('city')[col].shift(lag)
    for window in roll_windows:
        df[f'{col}_rollmean{window}'] = df.groupby('city')[col].transform(lambda x: x.rolling(window).mean())

# Sort by time
df.sort_values(by=['city', 'week_start_date'], ascending=[False, True], inplace=True)

/tmp/ipykernel_42921/3735553132.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_rollmean{window}'] = df.groupby('city')[col].transform(lambda x: x.rolling(window).mean())
/tmp/ipykernel_42921/3735553132.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_lag{lag}'] = df.groupby('city')[col].shift(lag)
/tmp/ipykernel_42921/3735553132.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining 

# Train and test

In [6]:
df = impute_with_mean(df, "total_cases")

df = removal_nonnumeric_columns(df)

train_X, test_X, train_y, test_y,validation = split_data(df)

model = train_model(train_X, train_y)

current_MAE = test_model(model, test_X, test_y)
# baseline: 14.629001914370031
# with new features: 11.263847476740525

Mean absolute Error is 14.140981322443796


FileNotFoundError: [Errno 2] No such file or directory: 'logs/MAEs.csv'

In [7]:
#Submission
X = remerge(train_X, test_X)
y = remerge(train_y, test_y)

final_model = train_model(X, y)
predictions = make_prediction(validation, final_model)

create_submission(predictions, validation, "lag_roll_features_30_04_1816.csv")


Submission file created at lag_roll_features_30_04_1816.csv
